<a href="https://colab.research.google.com/github/ben-ogden/pinecone-examples/blob/main/hello_serverless_grpc_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello, Pinecone Serverless!

This notebook will walk through the steps to get a simple Pinecone Serverless index up and running, using the grpc client.


## Prerequisites

Install dependencies.

In [98]:
!pip install -qU \
  "pinecone-client[grpc]"==3.0.0.dev4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.1 MB/s eta 0:00:00


Set up Pinecone. Get your Pinecone API key [here](https://www.pinecone.io/start).

In [99]:
import os
#from pinecone import Pinecone
# Everything grpc is now in this pinecone.grpc subpackage
from pinecone.grpc import PineconeGRPC as Pinecone
from google.colab import userdata

pinecone = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

## Pinecone quickstart

With Pinecone you can create a vector index where you can store and search through your vectors.

In [87]:
# Giving our index a name
index_name = "hello-serverless"

Creating a Pinecone Index.

In [101]:
import time
from pinecone import ServerlessSpec

# A new property 'spec' is used to tell Pinecone how we should deploy your index.
pinecone.create_index(
  name=index_name,
  dimension=3,
  metric="cosine",
  spec=ServerlessSpec(cloud='aws', region='us-west-2')
)

# wait for index to be ready before connecting
while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)

# Target your new serverless index.
idx = pinecone.Index(index_name)

# Now, use Python client methods as usual to perform data operations.
idx.describe_index_stats()


{'dimension': 3,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

We have the index ready. Now we will create some simple vectors that will serve as our examples.

In [102]:
import pandas as pd

df = pd.DataFrame(
    data={
        "id": ["A", "B"],
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

,id,vector
0,A,"[1.0, 1.0, 1.0]"
1,B,"[1.0, 2.0, 3.0]"


We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [103]:
idx.upsert(
    vectors=zip(df.id, df.vector),
    namespace='test-namespace'
)

upserted_count: 2

In [110]:

# wait for index to be ready before connecting
while not idx.describe_index_stats()['namespaces']:
    time.sleep(1)

idx.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'namespaces': {'test-namespace': {'vector_count': 2}},
 'total_vector_count': 2}

In [111]:
idx.query(
    vector=[2., 2., 2.],
    namespace='test-namespace',
    top_k=5,
    include_values=True) # returns top_k matches

{'check_type': False,
 'matches': [{'id': 'A',
              'metadata': {},
              'score': 1.0,
              'sparse_values': {'indices': [], 'values': []},
              'values': [1.0, 1.0, 1.0]},
             {'id': 'B',
              'metadata': {},
              'score': 0.9258201,
              'sparse_values': {'indices': [], 'values': []},
              'values': [1.0, 2.0, 3.0]}],
 'namespace': 'test-namespace',
 'results': []}

## Delete the Index
Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [100]:
pinecone.delete_index(index_name)